In [10]:
# for degradation experiment in directory take the first csv file with resnet in title
# that is, for degradation in exp in directory that contains either the term contrast, lowpass, highpass, or noise
import os, errno
import re
from csv import DictReader
from image_manipulation import load_image, contrast, noise, highpass, lowpass, save_img
from skimage.color import rgb2gray
from scipy.ndimage.filters import gaussian_filter
from scipy import fftpack as fp
from skimage.io import imread, imsave
from PIL import Image
import numpy as np
import pandas as pd
import sys
import time
from pathlib import Path

experiments = ['contrast', 'noise', 'highpass', 'lowpass']
experiments = [word + "-experiment" for word in experiments]
path = "../raw-data/TF"

def find_treated_images():
    """Finds the images that have been used by resnet for predictions.
    Only session 1 is used respectively. The function returns a list of the images.
    Each element is itself a list with: 
    - the complete image name with condition, 
    - object category (ground truth), 
    - experiment type,
    - experiment condition (level),
    - object response
    - raw image name (as used in ImageNet)
    """
    treated_images = []
    for directory in os.listdir(path):
        if directory in experiments:
            print(directory)
            # take the first csv file with resnet in title
            csv_files = os.listdir(os.path.join(path, directory))
            for csv_file in csv_files:
                if "resnet" in csv_file and "session_1" in csv_file:
                    file = os.path.join(path, directory, csv_file)
                    with open(file, 'r') as read_obj:
                        csv_dict_reader = DictReader(read_obj)
                        # for line in csv file get imagename, condition, experiment, and response
                        for row in csv_dict_reader:
                            img = row['imagename'].split("_")[-2:]
                            img = "_".join(img)
                            lst = [row['imagename'], row['category'], directory.replace("-experiment", ""), row['condition'], row['object_response'], img]
                            treated_images.append(lst)
    return treated_images

treated_images = find_treated_images()             

print(treated_images[0:6])
print(len(treated_images)) # 5120

def existence_treated_images():
    """ checks whether the treated images exist in our library
    takes as input the raw image name (e.g. n03041632_43625.JPEG)
    """
    pass

# cwd = Path.cwd()
# print(cwd)
# path = Path.cwd() / "treated_images"
out_dir = "treated_images/"
# if not os.path.exists(path):
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print("dir made")

#apply image degradation:
start_time = time.time()
counter = 0
dc_truth_response = dict()
unique_imgnames = []
for image in treated_images:
    # if image does not already exist in target directory do the following #TODO
    # print(image)
    out_dir = "treated_images/"
    exp = "sub"
    cond = "subsub"
    raw_img_name = image[-1]
    if not raw_img_name in unique_imgnames:
        unique_imgnames.append(raw_img_name)
    # if raw img name end with file ending .png and cannot be found FileNotFoundError,
    # turn it to jpeg and try again
    # print(raw_img_name)
    img = load_image(raw_img_name)
    try:
        if image[2] == "contrast":
            lvl = float(image[3].strip("c")) * 0.01
            img = contrast(img, lvl)
            image[3]
            exp = "con"

        if image[2] == "noise":
            lvl = float(image[3].strip("nse"))
            img = noise(img, lvl)
            exp = "nse"

        if image[2] == "highpass":
            lvl = image[3].strip("hp")
            if lvl == "inf":
                lvl = 999
            else:
                lvl = float(lvl)
            img = highpass(img, lvl)
            exp = "hp"

        if image[2] == "lowpass":
            lvl = float(image[3].strip("lp"))
            img = lowpass(img, lvl)
            exp = "lp"
        ground_truth = image[1]
        prediction = image[-2]
        cond = image[3]
        print(exp == "con", cond == "c100")
        print(exp, cond)
        
        image_file = f"{counter}_{exp}_dnn_{cond}_gt{ground_truth}_pred{prediction}_{raw_img_name}"
        dc_truth_response[f"{counter}"] = [counter, raw_img_name, exp, cond, ground_truth, prediction,                                                     ground_truth==prediction]
        # path =  f"{out_dir}{exp}/{cond}/"
        # out_dir = os.path.join(os.path.join(out_dir, f"{exp}/"), f"{cond}/")
        out_dir = out_dir + f"{exp}/{cond}/"
        # path = path / f"{exp}"
        print(out_dir)
        print(out_dir=="treated_images/con/c100/")
        # path = path / f"{cond}"
        # path = os.path.join(out_dir, exp, cond + "/")
        # print(path)
        if not os.path.exists(out_dir):
            print("does not exist")
            try:
                print("here")
                print(out_dir=="treated_images/con/c100/")
                os.makedirs(out_dir)
                print("dir made")
            except Exception as e:
                print(e)
            # print(image_file)
            # imgdata = np.asarray(img)
            save_img(img, out_dir + image_file, use_JPEG=False)
            print("processed image number", counter)
        counter += 1

    except:
        print("image could not be degraded, skipping image.")
        raise RuntimeError("image could not be degraded, skipping image.")

print("counter: ", counter)
print("--- %s seconds ---" % (time.time() - start_time))
print("len(dc_truth_response)", len(dc_truth_response))
print("len(unique_imgnames)", len(unique_imgnames))

contrast-experiment
highpass-experiment
lowpass-experiment
noise-experiment
[['0001_con_dnn_c100_knife_10_n03041632_43625.JPEG', 'knife', 'contrast', 'c100', 'knife', 'n03041632_43625.JPEG'], ['0002_con_dnn_c100_knife_10_n03041632_81943.JPEG', 'knife', 'contrast', 'c100', 'knife', 'n03041632_81943.JPEG'], ['0003_con_dnn_c100_knife_10_n03041632_12242.JPEG', 'knife', 'contrast', 'c100', 'knife', 'n03041632_12242.JPEG'], ['0004_con_dnn_c100_knife_10_n03041632_24384.JPEG', 'knife', 'contrast', 'c100', 'oven', 'n03041632_24384.JPEG'], ['0005_con_dnn_c100_knife_10_n03041632_24181.JPEG', 'knife', 'contrast', 'c100', 'knife', 'n03041632_24181.JPEG'], ['0006_con_dnn_c100_knife_10_n03041632_11773.JPEG', 'knife', 'contrast', 'c100', 'knife', 'n03041632_11773.JPEG']]
5120
True True
con c100
treated_images/con/c100/
does not exist
here
[WinError 3] Das System kann den angegebenen Pfad nicht finden: 'treated_images/con/c100/'
0_con_dnn_c100_gtknife_predknife_n03041632_43625.JPEG
image could not be d

RuntimeError: image could not be degraded, skipping image.

In [4]:
# count no of ground truth categories
print(df['ground_truth'].value_counts())

NameError: name 'df' is not defined

In [ ]:
print(len(dc_truth_response))
df = pd.DataFrame.from_dict(dc_truth_response, orient='index', columns=["counter", "raw_img_name", "exp",                                                       "cond", "ground_truth", "prediction", "agreement"])
print(df.head(10))
print(len(df))

In [ ]:
print(len(df))
print(df['agreement'].value_counts())

In [ ]:
# see absolute agreement for each category
df.groupby(["ground_truth", "agreement"]).size().reset_index(name='counts')

In [ ]:
df.groupby(["agreement", "ground_truth"]).size().reset_index(name='counts')